## 5.3 ラッキーセブン戦略の実装

### コード5.1 ライブラリの読み込みと入力データの準備

In [1]:
import pandas as pd
import statsmodels.api as sm

# 必要なデータの選択(行と列の選択)
# ticker末尾が0007の 2011/7-2014/6の36ヶ月のmonthとreturn
stockMonthly = pd.read_csv('./data/stockMonthly.csv', parse_dates=['month'])
stockMonthly['month'] = stockMonthly['month'].dt.to_period('M')
stockMonthly = stockMonthly[stockMonthly['ticker'].str[1:5] == '0007']
stockMonthly = stockMonthly[(stockMonthly['month'] >= '2011-7') &
                            (stockMonthly['month'] <= '2014-6')]
stockMonthly = stockMonthly[['month', 'ticker', 'return']]
print(stockMonthly)
##          month ticker      return
## 1596    2011-07  A0007    0.598007
## 1597    2011-08  A0007    2.245707

# 市場ポートフォリオの超過リターンが収録されたffMonthlyを読み込み
ffMonthly = pd.read_csv('./data/ffMonthly.csv', parse_dates=['month'])
ffMonthly['month'] = ffMonthly['month'].dt.to_period('M')
print(ffMonthly)
##        month   RMRF   SMB   HML    RF
## 0    1990-07  20.67 -1.56 -5.16  0.68
## 1    1990-08 -13.69 -3.63  0.98  0.66

          month ticker      return
1596    2011-07  A0007    0.598007
1597    2011-08  A0007    2.245707
1598    2011-09  A0007   -2.131783
1599    2011-10  A0007    4.818482
1600    2011-11  A0007   -3.211587
...         ...    ...         ...
561769  2014-02  Z0007  143.115500
561770  2014-03  Z0007   -7.559395
561771  2014-04  Z0007  -19.496360
561772  2014-05  Z0007   14.704930
561773  2014-06  Z0007   40.736580

[900 rows x 3 columns]
       month   RMRF   SMB   HML    RF
0    1990-07  20.67 -1.56 -5.16  0.68
1    1990-08 -13.69 -3.63  0.98  0.66
2    1990-09   0.63  5.18  1.83  0.60
3    1990-10  -3.76 -5.30  1.84  0.68
4    1990-11 -19.52 -0.50  1.43  0.57
..       ...    ...   ...   ...   ...
295  2015-02   4.59 -2.00  0.73  0.00
296  2015-03   8.81 -4.46  0.11  0.00
297  2015-04   0.99 -0.16 -1.50  0.00
298  2015-05   4.60 -0.98 -0.49  0.00
299  2015-06   2.66  0.49 -0.74  0.00

[300 rows x 5 columns]


### コード5.2 回帰モデル用のデータセット作成

In [2]:
# 抽出されたラッキーセブン銘柄群の等加重平均リターン（ラッキーセブン戦略によるポートフォリオリターン）を算定
df = stockMonthly.groupby('month').mean()
# ffMonthlyのRMRF,RFを結合(インデックスによる結合)
df = pd.merge(df, ffMonthly[['month', 'RMRF', 'RF']], on='month')
# ラッキーセブンポートフォリオの超過リターンを算定 (RPRF = return - RF)
df['RPRF'] = df['return'] - df['RF']
# 回帰モデル構築用のデータセットの完成
print(df)
##       month     return   RMRF    RF       RPRF
## 0   2011-07  11.665931   2.02  0.00  11.665931
## 1   2011-08   7.900743   3.99  0.01   7.890743

      month     return   RMRF    RF       RPRF
0   2011-07  11.665931   2.02  0.00  11.665931
1   2011-08   7.900743   3.99  0.01   7.890743
2   2011-09  -5.958109  -7.42  0.00  -5.958109
3   2011-10   3.969759   3.35  0.00   3.969759
4   2011-11   3.098784   4.33  0.00   3.098784
5   2011-12  -3.371601  -4.20  0.00  -3.371601
6   2012-01   3.790057  -2.81  0.00   3.790057
7   2012-02   5.814087   8.58  0.00   5.814087
8   2012-03   1.651638  -2.02  0.00   1.651638
9   2012-04   0.057895  -0.15  0.00   0.057895
10  2012-05  -2.748330   0.43  0.01  -2.758330
11  2012-06 -10.219615  -9.89  0.00 -10.219615
12  2012-07  17.613056   5.76  0.00  17.613056
13  2012-08  -5.565265  -2.39  0.01  -5.575265
14  2012-09  -3.461467   3.80  0.01  -3.471467
15  2012-10  -3.152734  -6.80  0.01  -3.162734
16  2012-11  10.036146   8.02  0.01  10.026146
17  2012-12   4.183837   4.73  0.01   4.173837
18  2013-01   0.351038   2.31  0.00   0.351038
19  2013-02  -1.057506   3.65  0.00  -1.057506
20  2013-03  

### コード5.3 回帰モデルの推定

In [3]:
# ラッキーセブン戦略によるJensenのアルファの推定
model = sm.OLS(df['RPRF'], sm.add_constant(df['RMRF']))
res = model.fit()  # OLSによる推定
print(res.summary())  # 推定結果を表示

                            OLS Regression Results                            
Dep. Variable:                   RPRF   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     46.79
Date:                Thu, 16 Dec 2021   Prob (F-statistic):           7.14e-08
Time:                        17:08:04   Log-Likelihood:                -100.27
No. Observations:                  36   AIC:                             204.5
Df Residuals:                      34   BIC:                             207.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6794      0.704      2.386      0.0